In [168]:
import pandas as pd
import pickle
import os

In [169]:
def EuclideanDistance(a,b):
    x = (a[0]-b[0])**2
    y = (a[1]-b[1])**2
    return (x+y)**(1/2)

In [184]:
# load original speed data
original_speed = pd.read_csv("link_speed.csv",header=None)
original_speed = original_speed[2]

In [170]:
# load road length file and join with id
link_file = pd.read_csv("link_file.csv",header=None)
road_file = pd.read_csv("road_file.csv",header=None)
road_file[3] = link_file[2]
road_file['speed'] = 0
road_file = road_file.rename(columns={0: "WISLR_LINKID", 1: "FROM", 2:'TO', 3:"LENGTH", "speed" :"SPEED"})

In [171]:
# read crash speed data
crash_speed = pd.read_csv(os.path.join("transportation_data","crash-data","crash-data-link-2017-2020-postspd.csv"))
crash_speed = crash_speed[["WISLR_LINKID","POSTSPD1"]]
crash_speed = crash_speed.rename(columns = {"POSTSPD1" : "SPEED"})
crash_speed = crash_speed.dropna()

In [172]:
# read coordinate data
with open("reference_coordinate", "rb") as f:
    coord = pickle.load(f)

In [173]:
road_coordinate = []
for index, row in road_file.iterrows():
    road_coordinate.append(coord[row["TO"]][:2])
road_file['coordinate'] = road_coordinate

In [174]:
speed = []
for i in road_file['WISLR_LINKID']:
    df = crash_speed[crash_speed['WISLR_LINKID'] == i]
    
    # check if there is a crash data on that road
    if df.shape[0] == 0:
        speed.append(0)
        continue
    
    # calculate average speed  
    speed.append(df['SPEED'].mean())

In [175]:
road_file['SPEED'] = speed

In [176]:
biggerThenZeroID = list(road_file[road_file['SPEED'] > 0]['WISLR_LINKID'])
biggerThenZeroCoord = list(road_file[road_file['SPEED'] > 0]['coordinate'])
biggerThenZeroSpeed = list(road_file[road_file['SPEED'] > 0]['SPEED'])

In [178]:
speed = []
for i in road_file['WISLR_LINKID']:
    row = road_file[road_file['WISLR_LINKID'] == i]
    spd = float(row['SPEED'].iloc[0])
    if spd > 0:
        speed.append(spd)
    else:
        currentCoord = row['coordinate'].iloc[0]
        distTemp = []
        for j in biggerThenZeroCoord:
            distTemp.append(EuclideanDistance(j,currentCoord))
        speed.append(biggerThenZeroSpeed[distTemp.index(max(distTemp))])
        
road_file["SPEED"] = speed

In [185]:
road_file["SPEED"] = (road_file["SPEED"] + original_speed)/2

In [188]:
road_file["TIME"] = road_file["LENGTH"]/road_file["SPEED"]

In [190]:
link_time = road_time[["FROM","TO","TIME"]]

NameError: name 'road_time' is not defined